## Importamos Pandas para trabajar con DataFrames

In [1]:
import pandas as pd

## Leemos el fichero en un DataFrame

In [3]:
df = pd.read_excel('2018-spring-match-data-OraclesElixir-2018-05-02.xlsx')

### Vemos qué columnas tiene

In [4]:
df.columns

Index(['gameid', 'url', 'league', 'split', 'date', 'week', 'game', 'patchno',
       'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1',
       'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'k', 'd', 'a',
       'teamkills', 'teamdeaths', 'doubles', 'triples', 'quadras', 'pentas',
       'fb', 'fbassist', 'fbvictim', 'fbtime', 'kpm', 'okpm', 'ckpm', 'fd',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elementals',
       'oppelementals', 'firedrakes', 'waterdrakes', 'earthdrakes',
       'airdrakes', 'elders', 'oppelders', 'herald', 'heraldtime', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochamps', 'dmgtochampsperminute', 'dmgshare',
       'earnedgoldshare', 'wards', 'wpm', 'wardshare', 'wardkills', 'wcpm',
       'visionwards', 'visionwardbuys', 'visiblewardclearrate',
       'invisiblewardclearrate', 'totalgold', 'earnedg

### Quitamos la liga China porque no da información

In [10]:
df2 = df.loc[df.league != 'LPL']

### Nos quedamos sólo con datos de los jugadores y no agregados de equipos

In [26]:
df3 = df2.loc[df.player != 'Team']

### Empezamos a quedarnos con lo que nos interesa

#### Debemos crear un nuevo identificador para evitar errores de IDs de partida repetidos en distintas ligas 

In [62]:
df3['gameid_league'] = df3.gameid.astype(str) + '_' + df3.league

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Creamos un diccionario con los picks de cada partida 

In [64]:
dict1 = {gameid_league: list(df3.loc[df3.gameid_league == gameid_league]['champion']) for gameid_league in df3.gameid_league.unique()}

#### Eliminamos el item nulo que no sabemos por qué se incluye

In [ ]:
import numpy as np

In [93]:
dict1.pop(np.NaN)

[]

#### Transformamos ese diccionario en un DataFrame

In [95]:
df4 = pd.concat([pd.DataFrame(dict1[gameid]).T for gameid in dict1.keys()])

#### Añadimos el gameId como columna

In [117]:
df4['gameid_league'] = dict1.keys()

#### Ordenamos las columnas y nos quedamos con las necesarias

In [118]:
df5 = df4[['gameid_league', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [109]:
df3.drop('gameid_league', inplace=True)

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [110]:
df3.ft = df3.ft.astype(int)

C:\Users\david\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


## Machine Learning

### Preparamos los datos para los modelos de aprendizaje automático

In [120]:
df_train = df3[['gameid_league', 'ft']].merge(df5, left_on='gameid_league', right_on='gameid_league')[::10].set_index('gameid_league')

### Separamos objetivo de variables de entrenamiento

In [122]:
y = df_train['ft']

#### Transformamos variables categóricas en dummies 

In [125]:
X = pd.get_dummies(df_train.drop('ft', axis=1))

### Empezamos a probar resultados con distintos modelos

#### SVC

In [135]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score

In [143]:
model1 = SVC()

In [146]:
y_pred1 = cross_val_predict(model1, X, y)

In [147]:
accuracy_score(y_true=y, y_pred=y_pred1)

0.5462459194776932

#### MLP

In [148]:
from sklearn.neural_network import MLPClassifier

In [149]:
model2 = MLPClassifier()

In [153]:
y_pred2 = cross_val_predict(model2, X, y)

C:\Users\david\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\david\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\david\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [154]:
accuracy_score(y_true=y, y_pred=y_pred2)

0.5244831338411317

#### NB

In [155]:
from sklearn.naive_bayes import BernoulliNB

In [156]:
model3 = BernoulliNB()

In [157]:
y_pred3 = cross_val_predict(model3, X, y)

In [158]:
accuracy_score(y_true=y, y_pred=y_pred3)

0.544069640914037

#### RF

In [159]:
from sklearn.ensemble import RandomForestClassifier

In [160]:
model4 = RandomForestClassifier()

In [161]:
y_pred4 = cross_val_predict(model4, X, y)

In [162]:
accuracy_score(y_true=y, y_pred=y_pred4)

0.5125136017410229

## Simulación

In [164]:
df_train['rf_pred'] = y_pred4

In [173]:
picks = ['Ornn', 'Trundle', 'Viktor', 'Kalista', 'Brand', 'Sion', 'Lee Sin', 'Ahri', 'Tristana', 'Braum']

In [168]:
model4.fit(X=X, y=y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [177]:
df_sim = pd.get_dummies(pd.DataFrame(picks))

## Prediccion resultados partidas 

In [185]:
fnatic = df.loc[df.team == 'Fnatic']

In [188]:
rng = df.loc[df.team == 'Royal Never Give Up']

In [192]:
df1 = df.loc[df.league != 'LPL']

In [209]:
df1 = df1.loc[df1.player == 'Team']

In [242]:
df2 = df1[['team', 'side', 'k', 'd', 'a', 'result', 'gameid',
       'teamkills', 'teamdeaths', 'doubles', 'triples', 'quadras', 'pentas',
       'fb', 'fbassist', 'fbvictim', 'fbtime', 'kpm', 'okpm', 'ckpm', 'fd',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elementals',
       'oppelementals', 'firedrakes', 'waterdrakes', 'earthdrakes',
       'airdrakes', 'elders', 'oppelders', 'herald', 'heraldtime', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochamps', 'dmgtochampsperminute', 'dmgshare',
       'earnedgoldshare', 'wards', 'wpm', 'wardshare', 'wardkills', 'wcpm',
       'visionwards', 'visionwardbuys', 'visiblewardclearrate',
       'invisiblewardclearrate', 'totalgold', 'earnedgpm', 'goldspent', 'gspd',
       'minionkills', 'monsterkills', 'monsterkillsownjungle',
       'monsterkillsenemyjungle', 'cspm', 'goldat10', 'oppgoldat10', 'gdat10',
       'goldat15', 'oppgoldat15', 'gdat15', 'xpat10', 'oppxpat10', 'xpdat10',
       'csat10', 'oppcsat10', 'csdat10', 'csat15', 'oppcsat15', 'csdat15']]

In [243]:
df3 = df2.groupby(['team', 'side'], as_index=False).mean()

In [244]:
df3.rename(columns={'result': 'winrate'}, inplace=True)

In [245]:
df1['team_side'] = df1['team'] + '_' + df1['side']

In [246]:
df3['team_side'] = df3['team'] + '_' + df3['side']

In [247]:
df4 = df1[['result','team_side', 'gameid']]
df5 = df4.merge(df3, left_on='team_side', right_on='team_side')

In [277]:
df5.sort_values(by='gameid', inplace=)

,result,team_side,gameid,team,side,k,d,a,winrate,teamkills,...,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
1805,0,Fenerbahce Esports_Red,210112,Fenerbahce Esports,Red,11.062500,10.687500,26.875000,0.375000,11.062500,...,404.937500,18879.812500,18668.750000,211.062500,327.187500,321.812500,5.375000,527.562500,518.187500,9.375000
1464,1,Team AURORA_Blue,210112,Team AURORA,Blue,11.961538,15.000000,30.000000,0.384615,11.961538,...,-657.769231,18611.346154,18628.769231,-17.423077,322.653846,316.230769,6.423077,515.076923,506.038462,9.038462
1806,0,Fenerbahce Esports_Red,210117,Fenerbahce Esports,Red,11.062500,10.687500,26.875000,0.375000,11.062500,...,404.937500,18879.812500,18668.750000,211.062500,327.187500,321.812500,5.375000,527.562500,518.187500,9.375000
1465,1,Team AURORA_Blue,210117,Team AURORA,Blue,11.961538,15.000000,30.000000,0.384615,11.961538,...,-657.769231,18611.346154,18628.769231,-17.423077,322.653846,316.230769,6.423077,515.076923,506.038462,9.038462
1623,0,Team AURORA_Red,210118,Team AURORA,Red,9.076923,11.384615,21.615385,0.384615,9.076923,...,-307.923077,18426.538462,18419.615385,6.923077,314.230769,311.769231,2.461538,508.615385,509.153846,-0.538462
460,1,Fenerbahce Esports_Blue,210118,Fenerbahce Esports,Blue,13.875000,9.687500,34.500000,0.625000,13.875000,...,873.062500,18675.812500,18211.687500,464.125000,323.875000,309.062500,14.812500,519.437500,502.750000,16.687500
1807,0,Fenerbahce Esports_Red,210119,Fenerbahce Esports,Red,11.062500,10.687500,26.875000,0.375000,11.062500,...,404.937500,18879.812500,18668.750000,211.062500,327.187500,321.812500,5.375000,527.562500,518.187500,9.375000
1466,1,Team AURORA_Blue,210119,Team AURORA,Blue,11.961538,15.000000,30.000000,0.384615,11.961538,...,-657.769231,18611.346154,18628.769231,-17.423077,322.653846,316.230769,6.423077,515.076923,506.038462,9.038462
1599,0,Dark Passage_Blue,210133,Dark Passage,Blue,8.615385,10.538462,20.769231,0.384615,8.615385,...,565.076923,18835.230769,18587.461538,247.769231,320.384615,328.230769,-7.846154,515.461538,523.000000,-7.538462
1442,1,YouthCREW_Red,210133,YouthCREW,Red,12.900000,13.850000,31.000000,0.550000,12.900000,...,-874.700000,18518.250000,18722.200000,-203.950000,315.100000,316.650000,-1.550000,496.500000,501.800000,-5.300000


In [269]:
df6 = df5.sort_values(by='gameid')[0::2].reset_index(drop=True)

In [270]:
df7 = df5.sort_values(by='gameid')[1::2].reset_index(drop=True)

In [271]:
df8 = pd.concat([df6, df7], axis=1)

In [275]:
df8[['result', 'side']]

,result,result,side,side
0,0,1,Red,Blue
1,0,1,Red,Blue
2,0,1,Red,Blue
3,0,1,Red,Blue
4,0,1,Blue,Red
5,1,0,Blue,Red
6,1,0,Blue,Red
7,0,1,Red,Blue
8,1,0,Red,Blue
9,0,1,Blue,Red
